In [4]:
import sqlite3
con = sqlite3.connect("products.db")
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS Pieces")
cur.execute("DROP TABLE IF EXISTS Providers")
cur.execute("DROP TABLE IF EXISTS Provides")

cur.executescript("""
CREATE TABLE Pieces (
 Code INTEGER NOT NULL,
 Name TEXT NOT NULL,
 PRIMARY KEY (Code)
 );
CREATE TABLE Providers (
 Code VARCHAR(40) NOT NULL,
 Name TEXT NOT NULL,
PRIMARY KEY (Code)
 );
CREATE TABLE Provides (
 Piece INTEGER,
 Provider VARCHAR(40),
 Price INTEGER NOT NULL,
 PRIMARY KEY(Piece, Provider)
 );


INSERT INTO Providers(Code, Name) VALUES('HAL','Clarke Enterprises');
INSERT INTO Providers(Code, Name) VALUES('RBT','Susan Calvin Corp.');
INSERT INTO Providers(Code, Name) VALUES('TNBC','Skellington Supplies');

INSERT INTO Pieces(Code, Name) VALUES(1,'Sprocket');
INSERT INTO Pieces(Code, Name) VALUES(2,'Screw');
INSERT INTO Pieces(Code, Name) VALUES(3,'Nut');
INSERT INTO Pieces(Code, Name) VALUES(4,'Bolt');

INSERT INTO Provides(Piece, Provider, Price) VALUES(1,'HAL',10);
INSERT INTO Provides(Piece, Provider, Price) VALUES(1,'RBT',15);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'HAL',20);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'RBT',15);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'TNBC',14);
INSERT INTO Provides(Piece, Provider, Price) VALUES(3,'RBT',50);
INSERT INTO Provides(Piece, Provider, Price) VALUES(3,'TNBC',45);
INSERT INTO Provides(Piece, Provider, Price) VALUES(4,'HAL',5);
INSERT INTO Provides(Piece, Provider, Price) VALUES(4,'RBT',7);
""")
con.commit()

In [5]:
# -- 5.1 Select the name of all the pieces.
cur.execute("SELECT Name FROM Pieces")
display(cur.fetchall())

[('Sprocket',), ('Screw',), ('Nut',), ('Bolt',)]

In [6]:
# -- 5.2  Select all the providers' data.
cur.execute("SELECT * FROM Providers")
display(cur.fetchall())

[('HAL', 'Clarke Enterprises'),
 ('RBT', 'Susan Calvin Corp.'),
 ('TNBC', 'Skellington Supplies')]

In [8]:
# -- 5.3 Obtain the average price of each piece (show only the piece code and the average price).
cur.execute("SELECT Piece, AVG(Price) FROM Provides GROUP BY Piece")
display(cur.fetchall())

[(1, 12.5), (2, 16.333333333333332), (3, 47.5), (4, 6.0)]

In [10]:
# -- 5.4  Obtain the names of all providers who supply piece 1.
cur.execute("""
    SELECT Providers.Name
    FROM Providers, Provides
    WHERE Providers.Code = Provides.Provider AND Provides.Piece = 1
""")
display(cur.fetchall())

[('Clarke Enterprises',), ('Susan Calvin Corp.',)]

In [11]:
# -- 5.5 Select the name of pieces provided by provider with code "HAL".
cur.execute("""
    SELECT Pieces.Name
    FROM Pieces, Provides
    WHERE Pieces.Code = Provides.Piece AND Provides.Provider = 'HAL'
""")
display(cur.fetchall())

[('Sprocket',), ('Screw',), ('Bolt',)]

In [12]:
# -- 5.6 For each piece, find the most expensive offering of that piece and include the piece name, provider name, and price (note that there could be two providers who supply the same piece at the most expensive price).
cur.execute("""
    SELECT Pieces.Name, Providers.Name, Provides.Price
    FROM Pieces, Providers, Provides
    WHERE Pieces.Code = Provides.Piece AND Providers.Code = Provides.Provider AND Provides.Price = (
        SELECT MAX(Price)
        FROM Provides
        WHERE Provides.Piece = Pieces.Code
    )
""")
display(cur.fetchall())


[('Sprocket', 'Susan Calvin Corp.', 15),
 ('Screw', 'Clarke Enterprises', 20),
 ('Nut', 'Susan Calvin Corp.', 50),
 ('Bolt', 'Susan Calvin Corp.', 7)]

In [13]:
# -- 5.7 Add an entry to the database to indicate that "Skellington Supplies" (code "TNBC") will provide sprockets (code "1") for 7 cents each.
cur.execute("""
    INSERT INTO Provides(Piece, Provider, Price) VALUES(1,'TNBC',7)
""")
con.commit()

In [15]:
# -- 5.8 Increase all prices by one cent.
cur.execute("""
    UPDATE Provides
    SET Price = Price + 1
""")
con.commit()

In [16]:
# -- 5.9 Update the database to reflect that "Susan Calvin Corp." (code "RBT") will not supply bolts (code 4).
cur.execute("""
    DELETE FROM Provides
    WHERE Piece = 4 AND Provider = 'RBT'
""")
con.commit()

In [17]:
# -- 5.10 Update the database to reflect that "Susan Calvin Corp." (code "RBT") will not supply any pieces (the provider should still remain in the database).
cur.execute("""
    DELETE FROM Provides
    WHERE Provider = 'RBT'
""")
con.commit()